In [5]:
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from scipy.stats import multivariate_normal
import numpy as np
import random as rd

First need to create k Gaussians for a mixture

In [6]:
# Create a synthetic dataset with 3 classes with predefined means and covariances
def createSyntheticData(means, covMats, priors, n):
    data = []
    for i in range(n):
        r = rd.random()
        if r < priors[0]:
            data.append(np.random.multivariate_normal(means[0], covMats[0]))
        elif r < priors[0] + priors[1]:
            data.append(np.random.multivariate_normal(means[1], covMats[1]))
        else:
            data.append(np.random.multivariate_normal(means[2], covMats[2]))
    return np.array(data)

# Create some new very distinct means and covariances
originalMeans = [[0,0], [10,10], [20,20]]
originalCovMats = [[[1,0],[0,1]], [[1,0],[0,1]], [[1,0],[0,1]]]
originalPriors = [1/3, 1/3, 1/3]
n = 1000
data = createSyntheticData(originalMeans, originalCovMats, originalPriors, n)
XTrain, XTest = train_test_split(data)

In [7]:
k = 3
kmeans = KMeans(n_clusters = k, max_iter= 1)
kmeans.fit(XTrain)

pointsInEachCluster = [XTrain[kmeans.labels_ == i] for i in range(k)]
print(len(pointsInEachCluster[0]))
print(len(pointsInEachCluster[1]))
print(len(pointsInEachCluster[2]))

centers = kmeans.cluster_centers_
means = []
covMats = []
for i in range(len(pointsInEachCluster)):
    points = pointsInEachCluster[i]
    means.append(centers[i])
    covMat = np.cov(points.T)
    covMats.append(covMat)

print(means[0], covMats[0])

prior = 1 / len(means)
priors = [prior for i in range(len(means))]

c:\Users\unova\anaconda3\envs\nb\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


258
261
231
[9.93426952 9.92103233] [[1.02265631 0.0359046 ]
 [0.0359046  1.02544892]]


In [8]:
# def multivariate_normal(x, mean, cov):
#     d = x.shape[0]
#     det = np.linalg.det(cov)
#     inv = np.linalg.inv(cov)
#     exponent = -0.5 * np.matmul(np.matmul((x - mean).T, inv), (x - mean))
#     coefficient = 1 / ((2 * np.pi) ** (d / 2) * np.sqrt(det))
#     return coefficient * np.exp(exponent)


In [9]:
# multivariate_normal(np.array([1, 2]), np.array([1, 2]), np.array([[1, 0], [0, 1]]))

In [10]:
# I need take the current values of means, covariance and priors and calculate the h_ti

# h_ti is the posterior probability of the ith data point belonging to the ith cluster for the t_th data point given the point
# and the previous values of the means, covariance and priors

# so just a normal distribution with the mean and covariance of the cluster


def getPosteriorForAllClusters(xt, currMeans, currCov, currPriors):
    posterior = []
    for i in range(len(currMeans)):
        mean = currMeans[i]
        cov = currCov[i]
        prior = currPriors[i]
        probab = multivariate_normal.pdf(xt, mean, cov)
        posterior.append(probab * prior)
    
    # normalize
    posterior = np.array(posterior)
    posterior = posterior / np.sum(posterior)

    return posterior


def getPosteriorForAllDataPoints(X, currMeans, currCov, currPriors):
    posterior = []
    for i in range(len(X)):
        xt = X[i]
        posterior.append(getPosteriorForAllClusters(xt, currMeans, currCov, currPriors))
    return np.array(posterior)

def getNewMeans(X, currMeans, posteriors):
    newMeans = []
    # use the conscise numpy way of doing this
    for i in range(len(currMeans)):
        mean = np.sum(posteriors[:, i].reshape(-1, 1) * X, axis = 0) / np.sum(posteriors[:, i])
        newMeans.append(mean)
    
    # This is just for verifying whether the conscise way copilot gave is correct and damn is it nice
    # newMeans2 = []
    # for i in range(len(currMeans)):
    #     s = 0
    #     for j in range(len(X)):
    #         s += posterior[j, i] * X[j]
    #     newMeans2.append(s / np.sum(posterior[:, i]))
    
    # print(newMeans2)
    
    return newMeans

def getNewCovMats(X, currMeans, posteriors):
    newCovMats = []
    for i in range(len(currMeans)):
        mean = currMeans[i]
        covMat = np.zeros((len(mean), len(mean)))
        for j in range(len(X)):
            diff = X[j] - mean
            diff = diff.reshape(-1, 1)
            covMat += posteriors[j, i] * np.matmul(diff, diff.T)
        covMat /= np.sum(posteriors[:, i])
        newCovMats.append(covMat)
    return newCovMats

def getNewPriors(X, currMeans, posteriors):
    newPriors = []
    for i in range(len(currMeans)):
        newPriors.append(np.sum(posteriors[:, i]) / len(X))
    return newPriors

def getNewParams(X, currMeans, currCov, currPriors):
    posteriors = getPosteriorForAllDataPoints(X, currMeans, currCov, currPriors)
    means = getNewMeans(X, currMeans, posteriors)
    covMats = getNewCovMats(X, currMeans, posteriors)
    priors = getNewPriors(X, currMeans, posteriors)
    return means, covMats, priors

for i in range(100):
    means, covMats, priors = getNewParams(XTrain, means, covMats, priors)

print(means)

[array([9.93426952, 9.92103233]), array([20.07936625, 20.04388117]), array([0.0499462 , 0.00616266])]


In [11]:
# Print which cluster each data point has most probability of belonging to
def getPredictions(X, currMeans, currCov, currPriors)->np.ndarray:
    print(currMeans)
    posterior = getPosteriorForAllDataPoints(X, currMeans, currCov, currPriors)
    predictions = (np.argmax(posterior, axis = 1))
    return predictions

print(getPredictions(XTest, means, covMats, priors))
print(kmeans.predict(XTest), "hi")

getPredictions(XTrain, means, covMats, priors) == kmeans.predict(XTrain)

[array([9.93426952, 9.92103233]), array([20.07936625, 20.04388117]), array([0.0499462 , 0.00616266])]
[1 1 0 2 2 0 0 2 2 1 2 2 2 2 1 2 0 2 2 2 1 1 2 1 1 2 1 1 2 1 1 2 2 1 1 1 0
 0 0 1 2 1 1 0 0 0 1 1 2 2 0 1 1 1 2 2 2 0 0 0 1 1 1 1 2 1 1 1 1 2 1 1 1 0
 2 0 2 0 1 0 2 0 0 1 2 0 1 2 2 1 1 0 2 1 2 1 2 0 2 0 2 1 1 2 2 1 1 1 1 0 2
 0 2 0 2 1 2 1 1 1 0 0 0 2 0 0 2 2 1 2 2 2 2 1 1 2 1 1 0 1 1 1 1 1 2 1 0 2
 1 1 0 2 1 2 1 1 0 2 2 1 0 2 1 1 1 2 2 0 0 1 2 1 0 0 2 1 0 1 0 1 1 0 1 2 1
 1 0 0 1 0 1 1 2 0 2 2 0 2 0 1 2 2 1 2 1 1 0 1 1 2 2 2 2 1 0 0 1 2 0 2 0 2
 2 1 2 2 0 1 2 2 2 2 0 0 1 1 2 1 0 0 1 0 1 2 2 2 0 2 0 0]
[1 1 0 2 2 0 0 2 2 1 2 2 2 2 1 2 0 2 2 2 1 1 2 1 1 2 1 1 2 1 1 2 2 1 1 1 0
 0 0 1 2 1 1 0 0 0 1 1 2 2 0 1 1 1 2 2 2 0 0 0 1 1 1 1 2 1 1 1 1 2 1 1 1 0
 2 0 2 0 1 0 2 0 0 1 2 0 1 2 2 1 1 0 2 1 2 1 2 0 2 0 2 1 1 2 2 1 1 1 1 0 2
 0 2 0 2 1 2 1 1 1 0 0 0 2 0 0 2 2 1 2 2 2 2 1 1 2 1 1 0 1 1 1 1 1 2 1 0 2
 1 1 0 2 1 2 1 1 0 2 2 1 0 2 1 1 1 2 2 0 0 1 2 1 0 0 2 1 0 1 0 1 1 0 1 2 1
 1 0 0 1 0 1 1 

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [12]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components = 3, max_iter = 100)
gmm.fit(XTrain)

print(gmm.means_)
gmm.predict(XTrain)

[[4.99462016e-02 6.16266113e-03]
 [2.00793662e+01 2.00438812e+01]
 [9.93426952e+00 9.92103233e+00]]


array([2, 2, 2, 1, 2, 2, 0, 2, 1, 0, 2, 2, 2, 1, 0, 1, 1, 1, 0, 2, 1, 1,
       2, 0, 1, 2, 1, 1, 2, 1, 1, 1, 0, 1, 2, 1, 2, 2, 2, 1, 2, 0, 2, 2,
       0, 1, 1, 2, 0, 0, 0, 2, 0, 1, 0, 0, 1, 1, 1, 0, 2, 2, 1, 1, 2, 2,
       1, 1, 2, 2, 2, 0, 0, 0, 2, 0, 0, 2, 2, 2, 0, 0, 0, 2, 1, 2, 2, 2,
       2, 1, 2, 1, 0, 1, 1, 2, 2, 2, 2, 1, 2, 1, 0, 1, 1, 1, 1, 2, 0, 0,
       2, 0, 0, 1, 0, 2, 2, 2, 1, 1, 0, 1, 0, 2, 1, 2, 2, 0, 1, 0, 2, 2,
       1, 2, 2, 1, 2, 2, 1, 2, 0, 1, 1, 0, 1, 2, 1, 1, 2, 1, 2, 2, 0, 2,
       1, 1, 2, 0, 1, 0, 1, 0, 0, 2, 1, 0, 1, 0, 2, 1, 0, 0, 1, 0, 0, 2,
       1, 2, 2, 2, 1, 2, 1, 1, 1, 0, 1, 0, 2, 0, 2, 1, 0, 2, 0, 1, 1, 1,
       2, 1, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 1, 2, 0, 2, 0, 1, 0, 1, 0,
       0, 0, 0, 2, 1, 1, 2, 1, 2, 0, 2, 0, 1, 0, 2, 1, 0, 0, 1, 1, 1, 0,
       0, 2, 0, 0, 2, 1, 1, 1, 1, 1, 2, 0, 0, 2, 1, 2, 1, 2, 0, 2, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 2, 1, 2, 2, 1, 1, 1, 0, 2, 1, 0, 1, 0, 1,
       2, 2, 0, 2, 0, 2, 1, 2, 0, 1, 1, 2, 2, 0, 0,

In [17]:
# Use iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Initialize the parameters randomly instead of kmeans
k = 3
means = []
covMats = []
for i in range(k):
    means.append(X[rd.randint(0, len(X)-1)])
    covMats.append(np.cov(X.T))

print(means)

# Initialize the priors uniformly
prior = 1 / len(means)
priors = [prior for i in range(len(means))]

for i in range(100):
    means, covMats, priors = getNewParams(X, means, covMats, priors)

print(means)

print(y)
print(getPredictions(X, means, covMats, priors))

[array([5.9, 3. , 5.1, 1.8]), array([6.8, 2.8, 4.8, 1.4]), array([7.7, 3.8, 6.7, 2.2])]
[array([6.11758764, 2.88281231, 4.84367433, 1.71893362]), array([5.45626353, 3.18753279, 2.52111943, 0.63605117]), array([6.43855402, 3.50749481, 4.00545376, 1.11297982])]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
[array([6.11758764, 2.88281231, 4.84367433, 1.71893362]), array([5.45626353, 3.18753279, 2.52111943, 0.63605117]), array([6.43855402, 3.50749481, 4.00545376, 1.11297982])]
[1 1 1 1 1 2 1 1 1 1 1 1 2 1 1 2 1 1 1 1 2 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1
 1 1 2 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 1 1 0
 1 1 1 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 2 1 0 0
 0 0 0 0 0 0 2 1 1 0 0 1 0 0 2 0 0 0 

It seems it is not a good idea to just directly compare using label no.
This is because the algorithm is only clustering, so if anything we should do a visualization comparison
In the above case for example, the first 20 or so points are the same and EM gets that as well. It's just that it labeled it as 1 instead of the original 0. Why? It just got to that cluster later that's it

So what did I learn?
Don't compare the label values in clustering algorithms directly with a labeled dataset
There's a reason it's called clustering instead of classification

Yep this was definitely worth the trouble. A lot of new useful knowledge and understanding has been gained here

(Also learned that None == [some list] gives [False for l in range(len(list))])

This was frustrating and useful at the same time